In [1]:
import os, glob
import json
import pandas as pd
import numpy as np

from sedpy.observate import Filter, load_filters
#import prospect

# Read in and Reformat Photometry

In [30]:
path = os.path.join(os.getcwd(), 'data', 'tde_catalog_v0.2.json')
with open(path, 'r') as f:
    j = json.load(f)

# get just the observational data for the host galaxies
# and fix the filter names for sedpy
obsdata = {}
filtermap = {'GALEX_FUV': 'galex_FUV',
             'GALEX_NUV': 'galex_NUV',
             'uvot_U': 'uvot_u',
             'uvot_B': 'uvot_b',
             'uvot_V': 'uvot_v'}

for tdename in j:
    out = j[tdename]['host']['observations']
    mask = []
    for i, filt in enumerate(out['filternames']):
        if filt in filtermap:
            out['filternames'][i] = filtermap[filt]
        if 'wise' in filt:
            mask.append(False)
        else:
            mask.append(True)
    out['photmask'] = mask
    
    obsdata[tdename] = out

In [31]:
obsdata['ASASSN-14ae']

{'maggies': [9.204999177369166e-10,
  1.6013266514117124e-09,
  3.095704037644609e-08,
  1.2320618180563926e-07,
  2.2101155291657063e-07,
  2.8343075871337945e-07,
  3.442112524116668e-07,
  2.435760381186772e-08,
  7.489698596424188e-08,
  1.5703475242774113e-07,
  2.3152799957554918e-07,
  1.3682429992529486e-07,
  1.0423034263532591e-07],
 'maggies_unc': [5.780764712529336e-10,
  6.742948174239415e-10,
  2.0999855050434808e-09,
  5.673854351807584e-09,
  1.0177958142503205e-08,
  1.3052468798188396e-08,
  1.5851513972878295e-08,
  1.3909192948688868e-09,
  4.138960401394744e-09,
  9.256598528566458e-09,
  1.0662258408667827e-08,
  6.300991867346606e-09,
  3.6152076563044984e-08],
 'mags': [22.589940614929805,
  21.988800170458678,
  18.773101415969176,
  17.27341875281689,
  16.638962559715072,
  16.368882551641452,
  16.157937341264482,
  19.033413594538484,
  17.813839147521556,
  17.01001056455924,
  16.588491201400757,
  17.159591913223267,
  17.45501458644867],
 'mags_unc': [0

In [32]:
outdir = os.path.join(os.getcwd(), 'data', 'hostdata')
for tdename in obsdata:
    outfile = os.path.join(outdir, f'{tdename}.json')
    j = json.dumps(obsdata[tdename], indent=4)
    with open(outfile, 'w') as f:
        f.write(j)

# Some code for prospector

In [4]:
run_params = {'verbose': True,
              'debug': False,
              'outfile': 'test',
              'output_pickles': False,
              # Optimization parameters
              'do_powell': False,
              'ftol': 0.5e-5, 'maxfev': 5000,
              'do_levenberg': True,
              'nmin': 10,
              # emcee fitting parameters
              'nwalkers': 128,
              'nburn': [16, 32, 64],
              'niter': 512,
              'interval': 0.25,
              'initial_disp': 0.1,
              # Obs data parameters
              'objid': 0,
              'phottable': 'demo_photometry.dat',
              'luminosity_distance': 1e-5,  # in Mpc
              # Model parameters
              'add_neb': False,
              'add_duste': False,
              # SPS parameters
              'zcontinuous': 1,
              }

In [5]:
def build_model(object_redshift=0.0, fixed_metallicity=None, add_duste=False,
                add_neb=False, luminosity_distance=0.0, **extras):
    """Construct a model.  This method defines a number of parameter
    specification dictionaries and uses them to initialize a
    `models.sedmodel.SedModel` object.

    :param object_redshift:
        If given, given the model redshift to this value.

    :param add_dust: (optional, default: False)
        Switch to add (fixed) parameters relevant for dust emission.

    :param add_neb: (optional, default: False)
        Switch to add (fixed) parameters relevant for nebular emission, and
        turn nebular emission on.

    :param luminosity_distance: (optional)
        If present, add a `"lumdist"` parameter to the model, and set it's
        value (in Mpc) to this.  This allows one to decouple redshift from
        distance, and fit, e.g., absolute magnitudes (by setting
        luminosity_distance to 1e-5 (10pc))
    """
    from prospect.models.templates import TemplateLibrary
    from prospect.models import priors, sedmodel

    # --- Get a basic delay-tau SFH parameter set. ---
    # This has 5 free parameters:
    #   "mass", "logzsol", "dust2", "tage", "tau"
    # And two fixed parameters
    #   "zred"=0.1, "sfh"=4
    # See the python-FSPS documentation for details about most of these
    # parameters.  Also, look at `TemplateLibrary.describe("parametric_sfh")` to
    # view the parameters, their initial values, and the priors in detail.
    model_params = TemplateLibrary["parametric_sfh"]

    # Add lumdist parameter.  If this is not added then the distance is
    # controlled by the "zred" parameter and a WMAP9 cosmology.
    if luminosity_distance > 0:
        model_params["lumdist"] = {"N": 1, "isfree": False,
                                   "init": luminosity_distance, "units":"Mpc"}

    # Adjust model initial values (only important for optimization or emcee)
    model_params["dust2"]["init"] = 0.1
    model_params["logzsol"]["init"] = -0.3
    model_params["tage"]["init"] = 13.
    model_params["mass"]["init"] = 1e8

    # If we are going to be using emcee, it is useful to provide an
    # initial scale for the cloud of walkers (the default is 0.1)
    # For dynesty these can be skipped
    model_params["mass"]["init_disp"] = 1e7
    model_params["tau"]["init_disp"] = 3.0
    model_params["tage"]["init_disp"] = 5.0
    model_params["tage"]["disp_floor"] = 2.0
    model_params["dust2"]["disp_floor"] = 0.1

    # adjust priors
    model_params["dust2"]["prior"] = priors.TopHat(mini=0.0, maxi=2.0)
    model_params["tau"]["prior"] = priors.LogUniform(mini=1e-1, maxi=10)
    model_params["mass"]["prior"] = priors.LogUniform(mini=1e6, maxi=1e10)

    # Change the model parameter specifications based on some keyword arguments
    if fixed_metallicity is not None:
        # make it a fixed parameter
        model_params["logzsol"]["isfree"] = False
        #And use value supplied by fixed_metallicity keyword
        model_params["logzsol"]['init'] = fixed_metallicity

    if object_redshift != 0.0:
        # make sure zred is fixed
        model_params["zred"]['isfree'] = False
        # And set the value to the object_redshift keyword
        model_params["zred"]['init'] = object_redshift

    if add_duste:
        # Add dust emission (with fixed dust SED parameters)
        model_params.update(TemplateLibrary["dust_emission"])

    if add_neb:
        # Add nebular emission (with fixed parameters)
        model_params.update(TemplateLibrary["nebular"])

    # Now instantiate the model using this new dictionary of parameter specifications
    model = sedmodel.SedModel(model_params)

    return model

In [12]:
def build_obs(objid=0, phottable='demo_photometry.dat',
              luminosity_distance=None, **kwargs):
    """Load photometry from an ascii file.  Assumes the following columns:
    `objid`, `filterset`, [`mag0`,....,`magN`] where N >= 11.  The User should
    modify this function (including adding keyword arguments) to read in their
    particular data format and put it in the required dictionary.

    :param objid:
        The object id for the row of the photomotery file to use.  Integer.
        Requires that there be an `objid` column in the ascii file.

    :param phottable:
        Name (and path) of the ascii file containing the photometry.

    :param luminosity_distance: (optional)
        The Johnson 2013 data are given as AB absolute magnitudes.  They can be
        turned into apparent magnitudes by supplying a luminosity distance.

    :returns obs:
        Dictionary of observational data.
    """

    from prospect.utils.obsutils import fix_obs
    
    # Build output dictionary.
    obs = {}
    # This is a list of sedpy filter objects.    See the
    # sedpy.observate.load_filters command for more details on its syntax.
    obs['filters'] = load_filters(tde['filternames'])
    # This is a list of maggies, converted from mags.  It should have the same
    # order as `filters` above.
    obs['maggies'] = np.array(tde['maggies'])
    # HACK.  You should use real flux uncertainties
    obs['maggies_unc'] = np.array(tde['maggies_unc'])
    # Here we mask out any NaNs or infs
    #obs['phot_mask'] = None
    # We have no spectrum.
    obs['wavelength'] = None
    obs['spectrum'] = None

    # Add unessential bonus info.  This will be stored in output
    #obs['dmod'] = catalog[ind]['dmod']
    obs['objid'] = tdename

    # This ensures all required keys are present and adds some extra useful info
    obs = fix_obs(obs)

    return obs

In [13]:
def build_sps(zcontinuous=1, compute_vega_mags=False, **extras):
    from prospect.sources import CSPSpecBasis
    sps = CSPSpecBasis(zcontinuous=zcontinuous,
                       compute_vega_mags=compute_vega_mags)
    return sps

def build_noise(**extras):
    return None, None

def build_all(**kwargs):

    return (build_obs(**kwargs), build_model(**kwargs),
            build_sps(**kwargs), build_noise(**kwargs))


In [14]:
obs, model, sps, noise = build_all()

NameError: name 'fsps' is not defined

In [15]:
os.getenv('SPS_HOME')

'~/bin/fsps/'